In [43]:
execfile('../spark-scripts/split.py')
execfile('../spark-scripts/utils.py')
execfile('../spark-scripts/eval.py')
execfile('../spark-scripts/SAGHFunctions.py')
execfile('../spark-scripts/SAGHMain.py')

In [44]:
import json
import copy

conf = {}

conf['split'] = {}
conf['split']['reclistSize'] = 100
conf['split']['callParams'] = {}
conf['split']['excludeAlreadyListenedTest'] = True
conf['split']['name'] = 'SenzaRipetizioni_nuovoEval1total_1413851857'
conf['split']['minEventsPerUser'] = 5
conf['split']['inputData'] = 's3n://contentwise-research-poli/split22.split/SenzaRipetizioni_nuovoEval1total_1413851857/'
#conf['split']['inputData'] = 's3n://contentwise-research-poli/30musicdataset/newFormat/relations/sessions.idomaar'
conf['split']['bucketName'] = 'contentwise-research-poli'
conf['split']['percUsTr'] = 0.05
conf['split']['ts'] = int(0.75 * (1421745857 - 1390209860) + 1390209860) - 10000
conf['split']['minEventPerSession'] = 5
conf['split']['onlineTrainingLength'] = 5
conf['split']['GTlength'] = 1
conf['split']['minEventPerSessionTraining'] = 10
conf['split']['minEventPerSessionTest'] = 11
conf['split']['mode'] = 'session'
conf['split']['forceSplitCreation'] = False

conf['evaluation'] = {}
conf['evaluation']['metric'] = {}
conf['evaluation']['metric']['type'] = 'recall'
conf['evaluation']['metric']['prop'] = {}
conf['evaluation']['metric']['prop']['N'] = [1,2,5,10,15,20,25,50,100]
conf['evaluation']['name'] = 'recall@N'

conf['general'] = {}
conf['general']['clientname'] = "split22.split"
conf['general']['bucketName'] = 'contentwise-research-poli'
conf['general']['tracksPath'] = '30Mdataset/entities/tracks.idomaar.gz'

conf['algo'] = {}
conf['algo']['name'] = 'SAGH'
conf['algo']['props'] = {}
# ***** EXAMPLE OF CONFIGURATION *****#
conf['algo']['props']['numGH'] = 100
conf['algo']['props']['skipTh'] = 0
# ****** END EXAMPLE ****************#

In [7]:
artistLookupRDD = loadArtistLookup(conf)
train,test = loadDataset(conf)
artistLookupRDD.cache()
train.cache()
test.cache()

In [45]:
from os import path
basePath = path.join("s3n://", conf['general']['bucketName'], conf['general']['clientname'])
splitPath = path.join(basePath, conf['split']['name'])

numGHList = [100]
skipThList = [0]

for th in skipThList:
    conf['algo']['props']['skipTh'] = th
    batchTrainingRDD = (train
                        .flatMap(lambda x: ext(json.loads(x))).filter(lambda x: x[1] > th)
                        .map(lambda x: (int(x[0]), int(x[2])))
                        .cache())
    recReqRDD = parseRequests(artistLookupRDD, test, th, conf).cache()
    
    for numGH in numGHList:
        recJsonRdd = generateRecommendationsSAGH(batchTrainingRDD, recReqRDD, artistLookupRDD, test, numGH, conf)
        saveRecommendations(conf, recJsonRdd, overwrite=True)
        computeMetrics(conf)

split/giroCompletoTestMultipleConfs_excludeTrue/Rec/SAGH_10#0 successfully deleted.
Recommendations successfully written to s3n://contentwise-research-poli/split/giroCompletoTestMultipleConfs_excludeTrue/Rec/SAGH_10#0/recommendations
recall@N successfully written to s3n://contentwise-research-poli/split/giroCompletoTestMultipleConfs_excludeTrue/Rec/SAGH_10#0/recall@N/metrics
